In [36]:
import cv2
import os

# Function to Extract Frames from Video
def extract_frames(video_path, output_folder="../outputs/frames4/", frame_rate=10):
    os.makedirs(output_folder, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print(f"End of video or error reading frame {frame_count}.")
            break
        
        # Debugging: print frame count to check
        print(f"Processing frame {frame_count}...")

        if frame_count % frame_rate == 0:
            frame_name = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            print(f"Saving frame as {frame_name}")
            success = cv2.imwrite(frame_name, frame)
            if success:
                print(f"Saved: {frame_name}")
            else:
                print(f"Failed to save: {frame_name}")

        frame_count += 1

    cap.release()
    print(f"Frames saved in {output_folder}")

# Test with the correct path
video_path = r"C:\Users\Admin\Desktop\Jupyter\data\videos\video3.mp4"
extract_frames(video_path)


Processing frame 0...
Saving frame as ../outputs/frames4/frame_0.jpg
Saved: ../outputs/frames4/frame_0.jpg
Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Saving frame as ../outputs/frames4/frame_10.jpg
Saved: ../outputs/frames4/frame_10.jpg
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Saving frame as ../outputs/frames4/frame_20.jpg
Saved: ../outputs/frames4/frame_20.jpg
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Saving frame as ../outputs/frames4/fra

In [37]:
from ultralytics import YOLO

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = len(results[0].boxes)  # Count bounding boxes
    return vehicles

# Test Detection on One Frame
vehicle_count = detect_vehicles("../outputs/frames4/frame_100.jpg")
print(f"Detected {vehicle_count} vehicles in frame_100.jpg")


image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_100.jpg: 384x640 21 cars, 1 bus, 1 truck, 316.6ms
Speed: 31.5ms preprocess, 316.6ms inference, 30.4ms postprocess per image at shape (1, 3, 384, 640)
Detected 23 vehicles in frame_100.jpg


In [38]:
import pandas as pd
# Generate Metrics for All Frames
metrics = []
frame_folder = "../outputs/frames4/"
for frame in os.listdir(frame_folder):
    frame_path = os.path.join(frame_folder, frame)
    vehicle_count = detect_vehicles(frame_path)
    metrics.append({"Frame": frame, "Vehicle_Count": vehicle_count})

# Save Metrics to CSV
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv("../outputs/metrics4.csv", index=False)
print("Metrics saved to outputs/metrics4.csv")



image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_0.jpg: 384x640 2 persons, 28 cars, 3 trucks, 161.0ms
Speed: 3.0ms preprocess, 161.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_10.jpg: 384x640 1 person, 26 cars, 2 trucks, 154.6ms
Speed: 3.0ms preprocess, 154.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_100.jpg: 384x640 21 cars, 1 bus, 1 truck, 321.6ms
Speed: 8.0ms preprocess, 321.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_110.jpg: 384x640 22 cars, 1 bus, 1 truck, 163.4ms
Speed: 5.0ms preprocess, 163.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_120.jpg: 384x640 

In [39]:
import pandas as pd
import os
import math
import cv2
from ultralytics import YOLO

# Constants for speed calculation
FRAME_RATE = 10  # frames per second (adjust based on your video properties)
TIME_INTERVAL = 1 / FRAME_RATE  # seconds
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        # Calculate center of bounding box
        x_center = (box.xyxy[0][0] + box.xyxy[0][2]) / 2
        y_center = (box.xyxy[0][1] + box.xyxy[0][3]) / 2
        vehicles.append((x_center.item(), y_center.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Match positions between consecutive frames
def match_positions(previous_positions, current_positions):
    matched_positions = []
    for prev in previous_positions:
        if not current_positions:
            break
        # Find the closest match in current positions
        closest_match = min(current_positions, key=lambda curr: math.sqrt((curr[0] - prev[0])**2 + (curr[1] - prev[1])**2))
        matched_positions.append(closest_match)
    return matched_positions

# Calculate average speeds for all frames
def calculate_speeds_in_kmph(frames_folder, scaling_factor, time_interval):
    frames = sorted(os.listdir(frames_folder))  # Ensure frames are processed in order
    previous_positions = []  # Track positions of vehicles in previous frame
    speeds = []  # Store average speeds for each frame

    for i, frame in enumerate(frames):
        frame_path = os.path.join(frames_folder, frame)
        current_positions = detect_vehicles(frame_path)

        if i > 0:  # Calculate speed only after the first frame
            matched_positions = match_positions(previous_positions, current_positions)
            frame_speeds = []

            for prev, curr in zip(previous_positions, matched_positions):
                dx = curr[0] - prev[0]
                dy = curr[1] - prev[1]
                pixel_displacement = math.sqrt(dx**2 + dy**2)
                real_displacement = pixel_displacement * scaling_factor  # meters
                speed_mps = real_displacement / time_interval  # meters per second
                speed_kmph = speed_mps * 3.6  # Convert to km/h

                frame_speeds.append(speed_kmph)

            # Calculate average speed for the frame
            avg_speed = sum(frame_speeds) / len(frame_speeds) if frame_speeds else 0
            speeds.append(avg_speed)
        else:
            speeds.append(0)  # No speed for the first frame

        previous_positions = current_positions  # Update for next iteration

    return speeds

# Update CSV file with speeds
def update_csv_with_speeds(csv_path, frames_folder):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate speeds
    avg_speeds_kmph = calculate_speeds_in_kmph(frames_folder, SCALING_FACTOR, TIME_INTERVAL)

    # Add a new column for average speeds (km/h)
    metrics_df['Average_Speed_kmph'] = avg_speeds_kmph

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths
    frames_folder = "../outputs/frames4/"
    metrics_csv_path = "../outputs/metrics4.csv"

    # Ensure outputs folder and CSV exist
    if not os.path.exists(frames_folder):
        print(f"Error: Frames folder '{frames_folder}' not found.")
    elif not os.path.exists(metrics_csv_path):
        print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
    else:
        update_csv_with_speeds(metrics_csv_path, frames_folder)


Loaded metrics from ../outputs/metrics4.csv

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_0.jpg: 384x640 2 persons, 28 cars, 3 trucks, 125.6ms
Speed: 5.0ms preprocess, 125.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame: ../outputs/frames4/frame_0.jpg, Detected Vehicles: [(715.6387329101562, 396.32391357421875), (543.775146484375, 358.1458740234375), (590.9141845703125, 402.685791015625), (284.0640869140625, 362.3115234375), (635.4937744140625, 296.37298583984375), (704.892822265625, 340.48046875), (335.03564453125, 168.78797912597656), (285.16796875, 309.7751770019531), (420.2418518066406, 308.27667236328125), (595.3073120117188, 250.20106506347656), (511.4828796386719, 164.66073608398438), (514.1355590820312, 294.62603759765625), (434.78106689453125, 377.79052734375), (41.15031433105469, 177.9669189453125), (621.4540405273438, 189.45111083984375), (488.6098327636719, 200.39920043945312), (44.43946838378906, 91.64285278320312),

In [40]:
import pandas as pd
import os
import math
from ultralytics import YOLO

# Constants
SCALING_FACTOR = 0.05  # meters per pixel (adjust based on calibration)

# Load YOLO Model
model = YOLO("models/yolo11n.pt")

# Vehicle Detection Function
def detect_vehicles(image_path):
    results = model(image_path)
    vehicles = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
        vehicles.append((x1.item(), y1.item(), x2.item(), y2.item()))  # Convert to Python floats
    print(f"Frame: {image_path}, Detected Vehicles: {vehicles}")
    return vehicles

# Calculate Queue Length
def calculate_queue_length(vehicles, axis="y"):
    """
    Calculate queue length based on vehicle bounding boxes.
    
    Args:
        vehicles (list): List of bounding boxes [(x1, y1, x2, y2), ...].
        axis (str): 'x' for horizontal queue or 'y' for vertical queue.
        
    Returns:
        float: Queue length in meters.
    """
    if not vehicles:
        return 0  # No vehicles detected, queue length is 0
    
    if axis == "x":  # Horizontal queue
        queue_positions = [((x1 + x2) / 2) for x1, y1, x2, y2 in vehicles]  # X centers
    elif axis == "y":  # Vertical queue
        queue_positions = [((y1 + y2) / 2) for x1, y1, x2, y2 in vehicles]  # Y centers
    else:
        raise ValueError("Invalid axis. Use 'x' for horizontal or 'y' for vertical.")

    # Sort vehicle positions along the queue axis
    queue_positions.sort()

    # Calculate total pixel distance
    total_pixel_distance = 0
    for i in range(1, len(queue_positions)):
        total_pixel_distance += abs(queue_positions[i] - queue_positions[i - 1])

    # Convert to meters
    queue_length_meters = total_pixel_distance * SCALING_FACTOR
    return queue_length_meters

# Update CSV with Queue Length
def update_csv_with_queue_length(csv_path, frames_folder, axis="y"):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Calculate queue lengths for all frames
    queue_lengths = []
    for frame in sorted(os.listdir(frames_folder)):
        frame_path = os.path.join(frames_folder, frame)
        vehicles = detect_vehicles(frame_path)
        queue_length = calculate_queue_length(vehicles, axis=axis)
        queue_lengths.append(queue_length)
        print(f"Frame: {frame}, Queue Length: {queue_length:.2f} meters")

    # Add queue length column to the dataframe
    metrics_df['Queue_Length_meters'] = queue_lengths

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Paths
    frames_folder = "../outputs/frames4/"
    metrics_csv_path = "../outputs/metrics4.csv"

    # Ensure outputs folder and CSV exist
    if not os.path.exists(frames_folder):
        print(f"Error: Frames folder '{frames_folder}' not found.")
    elif not os.path.exists(metrics_csv_path):
        print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
    else:
        # Choose 'x' for horizontal queues or 'y' for vertical queues
        update_csv_with_queue_length(metrics_csv_path, frames_folder, axis="y")


Loaded metrics from ../outputs/metrics4.csv

image 1/1 c:\Users\Admin\Desktop\Jupyter\notebooks\..\outputs\frames4\frame_0.jpg: 384x640 2 persons, 28 cars, 3 trucks, 144.0ms
Speed: 2.0ms preprocess, 144.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame: ../outputs/frames4/frame_0.jpg, Detected Vehicles: [(663.6822509765625, 360.6478271484375, 767.59521484375, 432.0), (500.8038024902344, 316.8106384277344, 586.7464599609375, 399.48114013671875), (532.675048828125, 373.3716125488281, 649.1533813476562, 432.0), (238.701416015625, 318.3399353027344, 329.4267272949219, 406.28314208984375), (594.2093505859375, 259.8815612792969, 676.7781982421875, 332.8644104003906), (659.0503540039062, 301.1668395996094, 750.7352294921875, 379.7940673828125), (312.21856689453125, 147.32809448242188, 357.8526916503906, 190.24786376953125), (245.3452606201172, 275.7497863769531, 324.9906921386719, 343.8005676269531), (378.38702392578125, 271.31195068359375, 462.0966796875, 345.2413635

In [41]:
import pandas as pd

# Congestion Classification Function
def classify_congestion(queue_length):
    """
    Classify congestion levels based on queue length.
    
    Args:
        queue_length (float): Queue length in meters.
        
    Returns:
        str: Congestion level ('Low', 'Medium', 'High').
    """
    if queue_length <= 10:
        return "Low"
    elif 10 < queue_length <= 20:
        return "Medium"
    else:
        return "High"

# Update CSV with Congestion Levels
def update_csv_with_congestion_levels(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure 'Queue_Length_meters' column exists
    if 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: 'Queue_Length_meters' column not found in the CSV file.")
        return

    # Classify congestion levels for each frame
    metrics_df['Congestion_Level'] = metrics_df['Queue_Length_meters'].apply(classify_congestion)

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with congestion levels saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Path to the metrics CSV file
    metrics_csv_path = "../outputs/metrics4.csv"

    # Check if the file exists
    if not os.path.exists(metrics_csv_path):
        print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
    else:
        update_csv_with_congestion_levels(metrics_csv_path)


Loaded metrics from ../outputs/metrics4.csv
Updated metrics with congestion levels saved to ../outputs/metrics4.csv


In [42]:
import pandas as pd

# Traffic Density Calculation Function
def calculate_traffic_density(vehicle_count, queue_length):
    """
    Calculate traffic density as vehicles per meter.
    
    Args:
        vehicle_count (int): Number of vehicles detected in the frame.
        queue_length (float): Queue length in meters.
        
    Returns:
        float: Traffic density (vehicles per meter).
    """
    if queue_length == 0:  # Avoid division by zero
        return 0
    return vehicle_count / queue_length

# Update CSV with Traffic Density
def update_csv_with_traffic_density(csv_path):
    # Load the existing metrics CSV file
    metrics_df = pd.read_csv(csv_path)
    print(f"Loaded metrics from {csv_path}")

    # Ensure required columns exist
    if 'Vehicle_Count' not in metrics_df.columns or 'Queue_Length_meters' not in metrics_df.columns:
        print("Error: Required columns 'Vehicle_Count' and 'Queue_Length_meters' are missing.")
        return

    # Calculate traffic density for each frame
    metrics_df['Traffic_Density_vehicles_per_meter'] = metrics_df.apply(
        lambda row: calculate_traffic_density(row['Vehicle_Count'], row['Queue_Length_meters']),
        axis=1
    )

    # Save the updated CSV file
    updated_csv_path = csv_path
    metrics_df.to_csv(updated_csv_path, index=False)
    print(f"Updated metrics with traffic density saved to {updated_csv_path}")

# Main Execution
if __name__ == "__main__":
    # Path to the metrics CSV file
    metrics_csv_path = "../outputs/metrics4.csv"

    # Check if the file exists
    if not os.path.exists(metrics_csv_path):
        print(f"Error: Metrics CSV file '{metrics_csv_path}' not found.")
    else:
        update_csv_with_traffic_density(metrics_csv_path)


Loaded metrics from ../outputs/metrics4.csv
Updated metrics with traffic density saved to ../outputs/metrics4.csv


In [43]:
import pandas as pd

# Load data
metrics_csv_path = "../outputs/metrics4.csv"
data = pd.read_csv(metrics_csv_path)

# Inspect the data
print("Initial Data Overview:")
print(data.info())
print(data.head())

# Drop unnecessary columns (e.g., 'Weather_Condition' if removed)
data = data.drop(columns=["Frame"], errors="ignore")  # Remove 'Frame' if not needed

# Handle duplicates
data = data.drop_duplicates()

# Fill missing numerical values with the column mean
numeric_columns = ["Vehicle_Count", "Average_Speed_kmph", "Queue_Length_meters", 
                   "Traffic_Density_vehicles_per_meter"]
for col in numeric_columns:
    data[col] = data[col].fillna(data[col].mean())

# Verify the cleaned data
print("Cleaned Data:")
print(data.info())


Initial Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Frame                               50 non-null     object 
 1   Vehicle_Count                       50 non-null     int64  
 2   Average_Speed_kmph                  50 non-null     float64
 3   Queue_Length_meters                 50 non-null     float64
 4   Congestion_Level                    50 non-null     object 
 5   Traffic_Density_vehicles_per_meter  50 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 2.5+ KB
None
           Frame  Vehicle_Count  Average_Speed_kmph  Queue_Length_meters  \
0    frame_0.jpg             33            0.000000            16.873122   
1   frame_10.jpg             29           33.277434            16.371307   
2  frame_100.jpg             23           77.673135         

In [44]:
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = data.drop(columns=["Congestion_Level"])  # Features
y = data["Congestion_Level"]  # Target

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Check transformed data
print(f"Scaled Features Shape: {X_scaled.shape}")


Scaled Features Shape: (50, 4)


In [45]:
from sklearn.model_selection import train_test_split

# Split data into train (70%), test (15%), eval (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_test, X_eval, y_test, y_eval = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print summary
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")
print(f"Evaluation set: {X_eval.shape}, {y_eval.shape}")


Training set: (35, 4), (35,)
Testing set: (7, 4), (7,)
Evaluation set: (8, 4), (8,)


In [47]:
# Calculate the total number of samples
total_samples = len(X_scaled)

# Calculate the percentages
train_percentage = (len(X_train) / total_samples) * 100
test_percentage = (len(X_test) / total_samples) * 100
eval_percentage = (len(X_eval) / total_samples) * 100

# Print the results
print(f"Train set: {len(X_train)} samples ({train_percentage:.2f}%)")
print(f"Test set: {len(X_test)} samples ({test_percentage:.2f}%)")
print(f"Evaluation set: {len(X_eval)} samples ({eval_percentage:.2f}%)")


Train set: 35 samples (70.00%)
Test set: 7 samples (14.00%)
Evaluation set: 8 samples (16.00%)


In [66]:
import pandas as pd
import json
import os

# Paths to the metric files
csv_paths = [
    "../outputs/metrics.csv",
    "../outputs/metrics2.csv",
    "../outputs/metrics3.csv",
    "../outputs/metrics4.csv"
]

# JSON file path to save the summary
summary_json_path = "../outputs/summary.json"

# Function to calculate average vehicle count from a CSV file
def calculate_average_vehicle_count(csv_path):
    if not os.path.exists(csv_path):
        print(f"Error: File not found - {csv_path}")
        return None
    metrics_df = pd.read_csv(csv_path)
    if 'Vehicle_Count' not in metrics_df.columns:
        print(f"Error: 'Vehicle_Count' column not found in {csv_path}")
        return None
    return metrics_df['Vehicle_Count'].mean()

# Dictionary to store average vehicle counts for each file
averages = {}

# Calculate average for each file
for csv_path in csv_paths:
    avg_count = calculate_average_vehicle_count(csv_path)
    if avg_count is not None:
        averages[os.path.basename(csv_path)] = round(avg_count, 2)  # Store filename as key

# Save results to JSON
try:
    # Load existing JSON file if it exists
    if os.path.exists(summary_json_path):
        with open(summary_json_path, "r") as json_file:
            summary_data = json.load(json_file)
    else:
        summary_data = {}

    # Add calculated averages to the JSON object
    summary_data.update(averages)

    # Save back to the JSON file
    with open(summary_json_path, "w") as json_file:
        json.dump(summary_data, json_file, indent=4)

    print(f"Summary saved to {summary_json_path}:")
    print(json.dumps(summary_data, indent=4))

except Exception as e:
    print(f"Error saving JSON: {e}")


Summary saved to ../outputs/summary.json:
{
    "metrics.csv": 18.36,
    "metrics2.csv": 24.6,
    "metrics3.csv": 26.16,
    "metrics4.csv": 25.04
}


In [65]:
import pandas as pd

# Path to the metrics CSV file
csv_path = "../outputs/metrics4.csv"

# Load the CSV file
metrics_df = pd.read_csv(csv_path)

# Calculate the average vehicle count
average_vehicle_count = metrics_df['Vehicle_Count'].mean()

# Display the result
print(f"Average Vehicle Count: {average_vehicle_count:.2f}")


Average Vehicle Count: 25.04


In [4]:
import pandas as pd

# Replace 'path_to_your_dataset.csv' with the actual file path
data = pd.read_csv("../outputs/combined_traffic_data.csv")
print(data.isnull().values.any())
print(data.isnull().sum())
print(data.isnull().sum().sum())



False
Direction                             0
Traffic_Volume                        0
Average_Speed_kmph                    0
Queue_Length_meters                   0
Traffic_Density_vehicles_per_meter    0
dtype: int64
0
